In [ ]:
import pandas as pd
import requests
from tqdm import tqdm
tqdm.pandas()
import json

In [ ]:
# url with a placeholder for cursor
example_url_with_cursor = 'https://api.openalex.org/works?filter=abstract.search:renewable%20energy&per-page=200&cursor={}&mailto=kuntakim88@gmail.com'
cursor = '*'

In [ ]:
 # set cursor value and request page from OpenAlex
# url = example_url_with_cursor.format(cursor)
# print('\n' + url)
# page_with_results = requests.get(url).json()

In [ ]:
# !mkdir renewable_json

In [ ]:
# json으로 저장
# with open('./renewable_json/renewable_1.json','w') as f:
#     json.dump(page_with_results['results'],f)

In [ ]:
example_url_with_cursor.format(cursor)

'https://api.openalex.org/works?filter=abstract.search:renewable%20energy&per-page=200&cursor=*&mailto=kuntakim88@gmail.com'

In [ ]:
end = int(239325/200)+1 # 일일 10만개, 1회 200개 요청
start = 1196
print(start, end)

1000 1197


In [ ]:
for idx in tqdm(range(start,end)):
     # set cursor value and request page from OpenAlex
    url = example_url_with_cursor.format(cursor)
    page_with_results = requests.get(url).json()
    
    # json으로 저장
    with open(f'./renewable_json/renewable_{idx}.json','w') as f:
        json.dump(page_with_results,f)
    # next page
    cursor = page_with_results['meta']['next_cursor']

100%|██████████| 196/196 [08:07<00:00,  2.48s/it]


In [ ]:
# json 파일 리스트
file_list = glob.glob('./renewable_json/*.json')
len(file_list)
file_list

['./renewable_json\\renewable_0.json',
 './renewable_json\\renewable_1.json',
 './renewable_json\\renewable_2.json',
 './renewable_json\\renewable_3.json',
 './renewable_json\\renewable_4.json']

In [ ]:
# 여러 json 파일을 읽어서 result만 하나의 리스트로 저장하기
openalex_json = []
for file in tqdm(file_list):
    with open(file,'r') as f:
        temp_json = json.load(f)['results']
    openalex_json.extend(temp_json)

100%|██████████| 5/5 [00:00<00:00,  6.41it/s]


In [ ]:
openalex_json

In [ ]:
# pandas dataframe으로 변환
df = pd.DataFrame.from_dict(openalex_json)
df

,id,doi,title,display_name,relevance_score,publication_year,publication_date,ids,primary_location,type,...,best_oa_location,grants,referenced_works,related_works,ngrams_url,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date
0,https://openalex.org/W2089525884,https://doi.org/10.1126/science.1212741,Electrical Energy Storage for the Grid: A Batt...,Electrical Energy Storage for the Grid: A Batt...,3568.97170,2011,2011-11-18,{'openalex': 'https://openalex.org/W2089525884...,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,None,[],"[https://openalex.org/W1626854854, https://ope...","[https://openalex.org/W1581508683, https://ope...",https://api.openalex.org/works/W2089525884/ngrams,"{'The': [0, 66], 'increasing': [1], 'interest'...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2023, 'cited_by_count': 496}, {'year...",2023-05-06T15:54:59.571436,2016-06-24
1,https://openalex.org/W2153292828,https://doi.org/10.1109/tie.2006.878356,Power-Electronic Systems for the Grid Integrat...,Power-Electronic Systems for the Grid Integrat...,3006.73660,2006,2006-08-07,{'openalex': 'https://openalex.org/W2153292828...,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,"{'is_oa': True, 'landing_page_url': 'https://i...",[],"[https://openalex.org/W1530411902, https://ope...","[https://openalex.org/W1546007654, https://ope...",https://api.openalex.org/works/W2153292828/ngrams,"{'The': [0, 22], 'use': [1], 'of': [2, 24, 60,...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2023, 'cited_by_count': 48}, {'year'...",2023-04-29T08:24:04.635622,2016-06-24
2,https://openalex.org/W2132755931,https://doi.org/10.1109/tie.2006.881997,Overview of Control and Grid Synchronization f...,Overview of Control and Grid Synchronization f...,2829.83420,2006,2006-10-02,{'openalex': 'https://openalex.org/W2132755931...,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,"{'is_oa': True, 'landing_page_url': 'http://ci...",[],"[https://openalex.org/W1512838495, https://ope...","[https://openalex.org/W1824538328, https://ope...",https://api.openalex.org/works/W2132755931/ngrams,"{'Renewable': [0], 'energy': [1, 17, 34], 'sou...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2023, 'cited_by_count': 35}, {'year'...",2023-04-28T21:43:12.141316,2016-06-24
3,https://openalex.org/W1971144082,https://doi.org/10.1021/cr900354u,The Catalytic Valorization of Lignin for the P...,The Catalytic Valorization of Lignin for the P...,2778.20070,2010,2010-03-10,{'openalex': 'https://openalex.org/W1971144082...,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,None,[],"[https://openalex.org/W11043624, https://opena...","[https://openalex.org/W2129464682, https://ope...",https://api.openalex.org/works/W1971144082/ngrams,"{'Biomass': [0], 'is': [1, 84, 96], 'an': [2],...",https://api.openalex.org/works?filter=cites:W1...,"[{'year': 2023, 'cited_by_count': 72}, {'year'...",2023-04-28T23:08:02.526437,2016-06-24
4,https://openalex.org/W1987415646,https://doi.org/10.1016/s0960-8524(01)00118-3,Energy production from biomass (part 1): overv...,Energy production from biomass (part 1): overv...,2603.12100,2002,2002-05-01,{'openalex': 'https://openalex.org/W1987415646...,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,None,[],"[https://openalex.org/W2019521372, https://ope...","[https://openalex.org/W1900165426, https://ope...",https://api.openalex.org/works/W1987415646/ngrams,"{'The': [0, 72, 153, 191], 'use': [1], 'of': [...",https://api.openalex.org/works?filter=cites:W1...,"[{'year': 2023, 'cited_by_count': 84}, {'year'...",2023-05-01T13:15:59.583852,2016-06-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://openalex.org/W1981595620,https://doi.org/10.1016/j.rser.2010.01.020,Analysis of the assessment factors for renewab...,Analysis of the assessment factors for rene

In [ ]:
# 중복삭제 (id 컬럼 기준)
df = df.drop_duplicates(subset=['id']).reset_index(drop=True)
df

,id,doi,title,display_name,relevance_score,publication_year,publication_date,ids,primary_location,type,...,best_oa_location,grants,referenced_works,related_works,ngrams_url,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date
0,https://openalex.org/W2089525884,https://doi.org/10.1126/science.1212741,Electrical Energy Storage for the Grid: A Batt...,Electrical Energy Storage for the Grid: A Batt...,3568.97170,2011,2011-11-18,{'openalex': 'https://openalex.org/W2089525884...,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,None,[],"[https://openalex.org/W1626854854, https://ope...","[https://openalex.org/W1581508683, https://ope...",https://api.openalex.org/works/W2089525884/ngrams,"{'The': [0, 66], 'increasing': [1], 'interest'...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2023, 'cited_by_count': 496}, {'year...",2023-05-06T15:54:59.571436,2016-06-24
1,https://openalex.org/W2153292828,https://doi.org/10.1109/tie.2006.878356,Power-Electronic Systems for the Grid Integrat...,Power-Electronic Systems for the Grid Integrat...,3006.73660,2006,2006-08-07,{'openalex': 'https://openalex.org/W2153292828...,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,"{'is_oa': True, 'landing_page_url': 'https://i...",[],"[https://openalex.org/W1530411902, https://ope...","[https://openalex.org/W1546007654, https://ope...",https://api.openalex.org/works/W2153292828/ngrams,"{'The': [0, 22], 'use': [1], 'of': [2, 24, 60,...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2023, 'cited_by_count': 48}, {'year'...",2023-04-29T08:24:04.635622,2016-06-24
2,https://openalex.org/W2132755931,https://doi.org/10.1109/tie.2006.881997,Overview of Control and Grid Synchronization f...,Overview of Control and Grid Synchronization f...,2829.83420,2006,2006-10-02,{'openalex': 'https://openalex.org/W2132755931...,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,"{'is_oa': True, 'landing_page_url': 'http://ci...",[],"[https://openalex.org/W1512838495, https://ope...","[https://openalex.org/W1824538328, https://ope...",https://api.openalex.org/works/W2132755931/ngrams,"{'Renewable': [0], 'energy': [1, 17, 34], 'sou...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2023, 'cited_by_count': 35}, {'year'...",2023-04-28T21:43:12.141316,2016-06-24
3,https://openalex.org/W1971144082,https://doi.org/10.1021/cr900354u,The Catalytic Valorization of Lignin for the P...,The Catalytic Valorization of Lignin for the P...,2778.20070,2010,2010-03-10,{'openalex': 'https://openalex.org/W1971144082...,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,None,[],"[https://openalex.org/W11043624, https://opena...","[https://openalex.org/W2129464682, https://ope...",https://api.openalex.org/works/W1971144082/ngrams,"{'Biomass': [0], 'is': [1, 84, 96], 'an': [2],...",https://api.openalex.org/works?filter=cites:W1...,"[{'year': 2023, 'cited_by_count': 72}, {'year'...",2023-04-28T23:08:02.526437,2016-06-24
4,https://openalex.org/W1987415646,https://doi.org/10.1016/s0960-8524(01)00118-3,Energy production from biomass (part 1): overv...,Energy production from biomass (part 1): overv...,2603.12100,2002,2002-05-01,{'openalex': 'https://openalex.org/W1987415646...,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,None,[],"[https://openalex.org/W2019521372, https://ope...","[https://openalex.org/W1900165426, https://ope...",https://api.openalex.org/works/W1987415646/ngrams,"{'The': [0, 72, 153, 191], 'use': [1], 'of': [...",https://api.openalex.org/works?filter=cites:W1...,"[{'year': 2023, 'cited_by_count': 84}, {'year'...",2023-05-01T13:15:59.583852,2016-06-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://openalex.org/W1981595620,https://doi.org/10.1016/j.rser.2010.01.020,Analysis of the assessment factors for renewab...,Analysis of the assessment factors for rene

In [ ]:
# 초록 텍스트 복원(리스트형태에서 텍스트로)
def index_to_text(abstract_inverted_index):
    abstract_index = {}
    for k, vlist in abstract_inverted_index.items():
        for v in vlist:
            abstract_index[v] = k
    abstract = ' '.join(abstract_index[k] for k in sorted(abstract_index.keys()))
    return abstract

In [ ]:
# 초록 복원 테스트
index_to_text(df['abstract_inverted_index'][0])

'The increasing interest in energy storage for the grid can be attributed to multiple factors, including the capital costs of managing peak demands, the investments needed for grid reliability, and the integration of renewable energy sources. Although existing energy storage is dominated by pumped hydroelectric, there is the recognition that battery systems can offer a number of high-value opportunities, provided that lower costs can be obtained. The battery systems reviewed here include sodium-sulfur batteries that are commercially available for grid applications, redox-flow batteries that offer low cost, and lithium-ion batteries whose development for commercial electronics and electric vehicles is being applied to grid storage.'

In [ ]:
# 초록 복원 결과를 'abstract' 컬럼에 추가
df['abstract'] = df[~pd.isna(df['abstract_inverted_index'])]['abstract_inverted_index'].progress_map(lambda x:index_to_text(x))

100%|██████████| 1000/1000 [00:00<00:00, 8497.20it/s]


In [ ]:
df['abstract']

0      The increasing interest in energy storage for ...
1      The use of distributed energy resources is inc...
2      Renewable energy sources like wind, sun, and h...
3      Biomass is an important feedstock for the rene...
4      The use of renewable energy sources is becomin...
                             ...                        
995    Abstract By 2030, Korean government aims to in...
996    Abstract This paper investigates the potential...
997    Abstract In this study, a hybrid energy storag...
998    Renewable energy is not a viable option unless...
999    This article discusses the growing need of ene...
Name: abstract, Length: 1000, dtype: object

In [ ]:
df

,id,doi,title,display_name,relevance_score,publication_year,publication_date,ids,primary_location,type,...,grants,referenced_works,related_works,ngrams_url,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date,abstract
0,https://openalex.org/W2089525884,https://doi.org/10.1126/science.1212741,Electrical Energy Storage for the Grid: A Batt...,Electrical Energy Storage for the Grid: A Batt...,3568.97170,2011,2011-11-18,{'openalex': 'https://openalex.org/W2089525884...,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,[],"[https://openalex.org/W1626854854, https://ope...","[https://openalex.org/W1581508683, https://ope...",https://api.openalex.org/works/W2089525884/ngrams,"{'The': [0, 66], 'increasing': [1], 'interest'...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2023, 'cited_by_count': 496}, {'year...",2023-05-06T15:54:59.571436,2016-06-24,The increasing interest in energy storage for ...
1,https://openalex.org/W2153292828,https://doi.org/10.1109/tie.2006.878356,Power-Electronic Systems for the Grid Integrat...,Power-Electronic Systems for the Grid Integrat...,3006.73660,2006,2006-08-07,{'openalex': 'https://openalex.org/W2153292828...,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,[],"[https://openalex.org/W1530411902, https://ope...","[https://openalex.org/W1546007654, https://ope...",https://api.openalex.org/works/W2153292828/ngrams,"{'The': [0, 22], 'use': [1], 'of': [2, 24, 60,...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2023, 'cited_by_count': 48}, {'year'...",2023-04-29T08:24:04.635622,2016-06-24,The use of distributed energy resources is inc...
2,https://openalex.org/W2132755931,https://doi.org/10.1109/tie.2006.881997,Overview of Control and Grid Synchronization f...,Overview of Control and Grid Synchronization f...,2829.83420,2006,2006-10-02,{'openalex': 'https://openalex.org/W2132755931...,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,[],"[https://openalex.org/W1512838495, https://ope...","[https://openalex.org/W1824538328, https://ope...",https://api.openalex.org/works/W2132755931/ngrams,"{'Renewable': [0], 'energy': [1, 17, 34], 'sou...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2023, 'cited_by_count': 35}, {'year'...",2023-04-28T21:43:12.141316,2016-06-24,"Renewable energy sources like wind, sun, and h..."
3,https://openalex.org/W1971144082,https://doi.org/10.1021/cr900354u,The Catalytic Valorization of Lignin for the P...,The Catalytic Valorization of Lignin for the P...,2778.20070,2010,2010-03-10,{'openalex': 'https://openalex.org/W1971144082...,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,[],"[https://openalex.org/W11043624, https://opena...","[https://openalex.org/W2129464682, https://ope...",https://api.openalex.org/works/W1971144082/ngrams,"{'Biomass': [0], 'is': [1, 84, 96], 'an': [2],...",https://api.openalex.org/works?filter=cites:W1...,"[{'year': 2023, 'cited_by_count': 72}, {'year'...",2023-04-28T23:08:02.526437,2016-06-24,Biomass is an important feedstock for the rene...
4,https://openalex.org/W1987415646,https://doi.org/10.1016/s0960-8524(01)00118-3,Energy production from biomass (part 1): overv...,Energy production from biomass (part 1): overv...,2603.12100,2002,2002-05-01,{'openalex': 'https://openalex.org/W1987415646...,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,[],"[https://openalex.org/W2019521372, https://ope...","[https://openalex.org/W1900165426, https://ope...",https://api.openalex.org/works/W1987415646/ngrams,"{'The': [0, 72, 153, 191], 'use': [1], 'of': [...",https://api.openalex.org/works?filter=cites:W1...,"[{'year': 2023, 'cited_by_count': 84}, {'year'...",2023-05-01T13:15:59.583852,2016-06-24,The use of renewable energy sources is becomin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://openalex.org/W1981595620,https://doi